<a href="https://colab.research.google.com/github/plthiyagu/Personnel/blob/master/Incremental_Learning_With_NLP_and_creme_retrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [11]:
messages=pd.read_csv('https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/smsspamcollection/SMSSpamCollection',sep='\t',names=["label", "message"])
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
messages.shape

(5572, 2)

In [15]:
from sklearn.model_selection import train_test_split
message_train,message_test=train_test_split(messages)

In [16]:
message_train

,label,message
2251,ham,I am getting threats from your sales executive...
85,ham,"I call you later, don't have network. If urgnt..."
4302,ham,Are there TA jobs available? Let me know pleas...
2578,ham,Hey whats up? U sleeping all morning?
383,ham,Hey leave it. not a big deal:-) take care.
...,...,...
1547,ham,Shant disturb u anymore... Jia you...
5236,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...
680,ham,Missed your call cause I was yelling at scrapp...
1796,ham,I hope your alright babe? I worry that you mig...


In [17]:
# Convert dataframe to list of tuples
messages_train = message_train.to_records(index=False)
messages_test=message_test.to_records(index=False)

In [18]:
message_train

,label,message
2251,ham,I am getting threats from your sales executive...
85,ham,"I call you later, don't have network. If urgnt..."
4302,ham,Are there TA jobs available? Let me know pleas...
2578,ham,Hey whats up? U sleeping all morning?
383,ham,Hey leave it. not a big deal:-) take care.
...,...,...
1547,ham,Shant disturb u anymore... Jia you...
5236,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...
680,ham,Missed your call cause I was yelling at scrapp...
1796,ham,I hope your alright babe? I worry that you mig...


In [21]:
!pip install creme

     |████████████████████████████████| 1.1MB 2.6MB/s 
  Created wheel for mmh3: filename=mmh3-2.5.1-cp36-cp36m-linux_x86_64.whl size=37860 sha256=560ae1c45c2a2f4f6a24eb36d5dade716350f0e91591c00302b993c37e6a1b0d
  Stored in directory: /root/.cache/pip/wheels/38/b4/ea/6e4e321c625d3320c0c496bf4088371546d8fce5f1dd71b219
Successfully built mmh3


In [22]:
# Creating the pipeline
# 1st function is creating the TFIDF
# 2nd function is the naive bayes predictor
import math
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
import creme
model = compose.Pipeline(
    ('tokenize', feature_extraction.TFIDF(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1)))


In [23]:
from creme import metrics
metric=metrics.Accuracy()
# Training the model row by row
for label,sentence in messages_train:
    model = model.fit_one(sentence, label)
    y_pred = model.predict_one(sentence)
    metric = metric.update(label, y_pred)

In [24]:
### Training Data Accuracy
metric

Accuracy: 95.86%

In [25]:
### test Data Accuracy
test_metric=metrics.Accuracy()
for label,sentence in messages_test:
    y_pred = model.predict_one(sentence)
    test_metric = metric.update(label, y_pred)

In [26]:
### test Metric
test_metric

Accuracy: 95.80%

In [27]:
model.fit_one("This guy is neutral", "ham")
model.fit_one("Everybody is neutral ", "ham")

Pipeline (
  TFIDF (
    normalize=True
    on=None
    strip_accents=True
    lowercase=False
    preprocessor=None
    tokenizer=<built-in method findall of _sre.SRE_Pattern object at 0x7f6460afca58>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1
  )
)

In [28]:
import math
from creme import compose
from creme import feature_extraction
from creme import naive_bayes
import creme

In [29]:
docs = [   ('Chinese Beijing Chinese', 'yes'),
 ('Chinese Chinese Shanghai', 'yes'),
('Chinese Macao', 'yes'),
('Tokyo Japan Chinese', 'no')
]

In [30]:
### This is just an example
corpus = [
         'This is the first document.',
        'This document is the second document.',
        'And this is the third one.',
        'Is this the first document?',
    ]
bow = creme.feature_extraction.TFIDF()
for sentence in corpus:
    print(bow.transform_one(sentence))

{'this': 0.4472135954999579, 'is': 0.4472135954999579, 'the': 0.4472135954999579, 'first': 0.4472135954999579, 'document': 0.4472135954999579}
{'this': 0.35355339059327373, 'document': 0.7071067811865475, 'is': 0.35355339059327373, 'the': 0.35355339059327373, 'second': 0.35355339059327373}
{'and': 0.408248290463863, 'this': 0.408248290463863, 'is': 0.408248290463863, 'the': 0.408248290463863, 'third': 0.408248290463863, 'one': 0.408248290463863}
{'is': 0.4472135954999579, 'this': 0.4472135954999579, 'the': 0.4472135954999579, 'first': 0.4472135954999579, 'document': 0.4472135954999579}


In [31]:
model = compose.Pipeline(
    ('tokenize', feature_extraction.BagOfWords(lowercase=False)),
    ('nb', naive_bayes.MultinomialNB(alpha=1)))

In [32]:
%%time
for sentence, label in docs:
     model = model.fit_one(sentence, label)

CPU times: user 160 µs, sys: 24 µs, total: 184 µs
Wall time: 188 µs


In [33]:
new_unseen_text = 'Tokyo india'

Training on a new data and new category¶


In [34]:
model.fit_one('India USA','may be')

Pipeline (
  BagOfWords (
    on=None
    strip_accents=True
    lowercase=False
    preprocessor=None
    tokenizer=<built-in method findall of _sre.SRE_Pattern object at 0x7f6460afca58>
    ngram_range=(1, 1)
  ),
  MultinomialNB (
    alpha=1
  )
)

In [35]:
model.predict_one("India Germany")

'may be'

Creme With Logistic Regression¶


In [36]:
from creme import compose
from creme import datasets
from creme import linear_model
from creme import metrics
from creme import preprocessing

X_y = datasets.Phishing()

model = compose.Pipeline(
   preprocessing.StandardScaler(),
 linear_model.LogisticRegression()
)
metric = metrics.Accuracy()


for x, y in X_y:
    y_pred = model.predict_one(x)      # make a prediction
    metric = metric.update(y, y_pred)  # update the metric
    model = model.fit_one(x, y)        # make the model learn

metric

Accuracy: 89.20%